In [4]:
import tensorflow as tf

import tensorflow_datasets as tfds
import os

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
mnist_train, info = tfds.load('mnist', split = 'train', as_supervised=True, with_info=True)
mnist_test = tfds.load('mnist', split='test', as_supervised=True)

In [19]:
def compile_model(new_model):
    new_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    print(new_model.summary())
    return new_model

def create_model():
    new_model = tf.keras.Sequential([
        tf.keras.layers.InputLayer((28,28,1)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
    return compile_model(new_model)


In [20]:
def wrangle_data(dataset, split):
    wrangled = dataset.map(lambda img, lbl: (tf.cast(img, tf.float32)/255.0, lbl))
    wrangled = wrangled.cache()
    if split == 'train':
        wrangled = wrangled.shuffle(60000)
    return wrangled.batch(64).prefetch(tf.data.AUTOTUNE)


In [21]:
train_data = wrangle_data(mnist_train, 'train')
test_data = wrangle_data(mnist_test, 'test')
model = create_model()
history = model.fit(train_data, epochs=5)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 64)                50240     
                                                                 
 dense_1 (Dense)             (None, 10)                650       
                                                                 
Total params: 50,890
Trainable params: 50,890
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/5
938/938 [==============================] - 3s 2ms/step - loss: 0.3448 - accuracy: 0.9045
Epoch 2/5
938/938 [==============================] - 2s 2ms/step - loss: 0.1705 - accuracy: 0.9517
Epoch 3/5
938/938 [==============================] - 2s 2ms/step - loss: 0.1252 - accuracy: 0.9643
Epoch 4/5
938/

In [22]:
model.evaluate(test_data)

157/157 [==============================] - 1s 3ms/step - loss: 0.0992 - accuracy: 0.9696


[0.09916854649782181, 0.9696000218391418]

In [23]:
model.save('mnist.h5')